### Convert GeoJSON point x,y coordinates to circles for use in petro-image
##### By Glenn R. Sharman for use by the Clastic Stratigraphy Research Group, University of Arkansas
##### Updated May  9, 2025

In [ ]:
# Import required modules
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from PIL import Image
import transformFuncs as tFunc

In [ ]:
# The geojson with image coordinates of selected spots
input_geojson = 'example-data/FSW24-01_zr_sticky.geojson'

output_geojson = 'outputs/FWS24-01_zr_sticky_circles.geojson'

# The file path to the image you want to place circles on
input_image = 'example-data/FWS24-01 5x Zr sticky tape mount.jpg'

# The output file path for the image with circles on it
output_image = 'outputs/FWS24-01_zr_sticky tape mount points to circles.jpg'

# Output image resolution
dpi = 300

# Diameter of the circle you want to plot, in microns
diameter_um = 20
num_vertices = 25  # number of points around the circle

circle_color = '#000000'
circle_opacity = 0

##### Execute the code

In [ ]:
# Open and load the GeoJSON file
with open(input_geojson, 'r') as file:
    data = json.load(file)  # Parses JSON into a Python dictionary
df = pd.DataFrame(data['features'])

# Load x and y coordinates and labels into lists
x_px = []
y_px = []
labels = []
for i in range(len(df)):
    x_px.append(df.iloc[i]['geometry']['coordinates'][0])
    y_px.append(df.iloc[i]['geometry']['coordinates'][1])
    labels.append(df.iloc[i]['properties']['label'])

# Prepare GeoJSON structure
geojson = {
    "type": "FeatureCollection",
    "features": []  # Initialize as an empty list to store features
}

# Note: The properties of the first annotation in the input GeoJSON will be mimicked
pixels_per_meter = df.iloc[0]['properties']['pixelsPerMeter']
r = diameter_um/2 * pixels_per_meter / 1e6

# Export a GeoJSON with the results
for i in range(len(x_px)):
    # x,y coordinates of the circle
    angles = np.linspace(0, 2 * np.pi, num_vertices + 1)
    x_coords = x_px[i] + r * np.cos(angles)
    y_coords = y_px[i] + r * np.sin(angles)

    feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": 'Polygon',
                        "coordinates": list([list(zip(x_coords, y_coords))]) # coordinates_array  # Should wrap in an extra list for GeoJSON Polygon
                    },
                    "properties": {
                        "uuid": tFunc.generate_unique_id(16),
                        "label": '', # Make an empty label
                        #"label": df.iloc[i]['properties']['label'], # Use the existing label
                        "xLabel": x_px[i],
                        "yLabel": y_px[i],
                        "imageTitle": df.iloc[0]['properties']['imageTitle'],
                        "pixelsPerMeter": pixels_per_meter,
                        "imageWidth": df.iloc[0]['properties']['imageWidth'],
                        "imageHeight": df.iloc[0]['properties']['imageHeight'],
                        "labelFontSize": df.iloc[0]['properties']['labelFontSize'],
                        "labelFontColor": df.iloc[0]['properties']['labelFontColor'],
                        "labelBackgroundColor": df.iloc[0]['properties']['labelBackgroundColor'],
                        "labelBackgroundOpacity": df.iloc[0]['properties']['labelBackgroundOpacity'],
                        "lineWeight": df.iloc[0]['properties']['lineWeight'],
                        "lineColor": df.iloc[0]['properties']['lineColor'],
                        "lineOpacity": df.iloc[0]['properties']['lineOpacity'],
                        "fillColor": circle_color,
                        "fillOpacity": circle_opacity
                    }
                }
    geojson["features"].append(feature)
with open(output_geojson, "w") as geojson_file:
    json.dump(geojson, geojson_file, indent=2)

In [ ]:
# Open and load the image
Image.MAX_IMAGE_PIXELS = 2000000000 # To avoid errors that the files are too large
image = Image.open(input_image)

# Make and export the figure
fig, ax = plt.subplots(1, figsize=(image.size[0]/dpi,image.size
[1]/dpi))

for i in range(len(geojson['features'])):
    coordinates = geojson['features'][i]['geometry']['coordinates']
    ax.plot(np.array(coordinates)[0][:,0], np.array(coordinates)[0][:,1], '-', color='red')
    #ax.text(x=C_transformed[i][0]+adjust_px, y=C_transformed[i][1]-adjust_px, s=labels[i], fontsize='medium', color='red',)
ax.imshow(image, cmap='Greys_r')
#ax.legend()
plt.axis('off')

fig.savefig(output_image, dpi=dpi)